In [ ]:
## using pyaudio with portaudio

## https://realpython.com/playing-and-recording-sound-python/
## Playing and Recording Sound in Python

## python-sounddevice and pyaudio provide bindings for the PortAudio library for cross-platform playback
##      of WAV files.

In [ ]:
## anaconda environment setup

uses pyaudio and portaudio

		conda environment setup  -- using python 3.7.7 as thats what combined execution is using already
conda create -n ce10recsnd1 python=3.7.7
conda install pyaudio			-- installed portaudio automatically
conda install jupyter

# Check soundcard status

### Check for Soundcard with PyAudio when Soundcard is present or removed
### https://stackoverflow.com/questions/28453240/check-for-soundcard-with-pyaudio-when-soundcard-is-present-or-removed

In [1]:
import pyaudio

In [2]:
pa = pyaudio.PyAudio()
pa.get_default_output_device_info()

{'index': 11,
 'structVersion': 2,
 'name': 'default',
 'hostApi': 0,
 'maxInputChannels': 32,
 'maxOutputChannels': 32,
 'defaultLowInputLatency': 0.008684807256235827,
 'defaultLowOutputLatency': 0.008684807256235827,
 'defaultHighInputLatency': 0.034807256235827665,
 'defaultHighOutputLatency': 0.034807256235827665,
 'defaultSampleRate': 44100.0}

In [ ]:
## never worked and showed anything - even when the SETTINGS in ubuntu showed fine
"""
def get_soundcard_dev_info():
    pad_sc = pyaudio.PyAudio()
    max_devs = pad_sc.get_device_count()
    input_devices_index = []
    output_devices_index = []
    print(f"max_devs = {max_devs}")
    
    for i in range(max_devs):
        devinfo = pad_sc.get_device_info_by_index(i)
        if "TUSBAudio ASIO Driver" in devinfo['name']:
            input_devices_index.append(int(devinfo['index']))
            output_devices_index.append(int(devinfo['index']))
        if not input_devices_index:
            print(f"NONE")
        print("f{input_devices_index}")
        pad_sc.terminate()
"""
get_soundcard_dev_info()

# Play wav file with pyaudio
### pyaudio provides bindings for PortAudio, the cross-platform audio I/O library. This means that you can use pyaudio to play and record audio on a variety of platforms, including Windows, Linux, and Mac. With pyaudio, playing audio is done by writing to a .Stream

### As you may have noticed, playing sounds with pyaudio is a bit more complex than playing sounds with the libraries you’ve seen earlier. This means that it may not be your first choice if you just want to play a sound effect in your Python application. However, because pyaudio gives you more low-level control, it is possible to get and set parameters for your input and output devices, and to check your CPU load and input or output latency. It also allows you to play and record audio in callback mode, where a specified callback function is called when new data is required for playback, or available for recording. These options make pyaudio a suitable library to use if your audio needs go beyond simple playback. Now that you’ve seen how you can use a number of different libraries to play audio, it’s time to see how you can use Python to record audio yourself.

In [22]:
import pyaudio
import wave
import os

In [4]:
#playFile = r'/media/rohit/DATA/EverythingD/01SRH-BDBA Acads/Thesis/StoryGenerator/audio/wavs/st_1_HM_Rohit_file1.wav'
PLAY_FILE = r'/home/rohit/PyWDUbuntu/thesis/audio/wavs/pyaudio_rec_3.wav'

In [5]:
os.path.exists(PLAY_FILE)

True

In [6]:
if os.path.exists(PLAY_FILE):
    # Set chunk size of 1024 samples per data frame
    chunk = 1024  

    # Open the sound file 
    wf = wave.open(PLAY_FILE, 'rb')

    # Create an interface to PortAudio
    p = pyaudio.PyAudio()

    # Open a .Stream object to write the WAV file to
    # 'output = True' indicates that the sound will be played rather than recorded
    stream = p.open(format = p.get_format_from_width(wf.getsampwidth()),
                    channels = wf.getnchannels(),
                    rate = wf.getframerate(),
                    output = True)

    # Read data in chunks
    data = wf.readframes(chunk)

    # Play the sound by writing the audio data to the stream
    ###########
    ## without checking for data = b'' it never exits so changed as per this link:
    # https://stackoverflow.com/questions/34856992/when-i-play-a-wav-in-python-the-program-doesnt-stop-when-the-file-ends-leavin
    #while data != '':
    #    stream.write(data)
    #    data = wf.readframes(chunk)
    ###########
    while data != b'':
        stream.write(data)
        data = wf.readframes(chunk)
    # Close and terminate the stream
    stream.close()
    p.terminate()
else:
    print(f"File not found: {PLAY_FILE}")

print(f"\nDone")


Done


In [ ]:
listen_for_secs = 5
chunk = 1024  # Record in chunks of 1024 samples
sample_format = pyaudio.paInt16  # 16 bits per sample
channels = 1 # mono
fs = 16000  # Record at 16kHz

p = pyaudio.PyAudio()  # Create an interface to PortAudio

print('Recording')

## input = True means record
stream = p.open(format=sample_format,
                channels=channels,
                rate=fs,
                frames_per_buffer=chunk,
                input=True)

frames = []  # Initialize array to store frames

# Store data in chunks for 3 seconds
for i in range(0, int(fs / chunk * listen_for_secs)):
    data = stream.read(chunk)
    frames.append(data)

# Stop and close the stream 
stream.stop_stream()
stream.close()
# Terminate the PortAudio interface
p.terminate()

print('Finished recording')

# Save the recorded data as a WAV file
wf = wave.open(sav2File, 'wb')
wf.setnchannels(channels)
wf.setsampwidth(p.get_sample_size(sample_format))
wf.setframerate(fs)
wf.writeframes(b''.join(frames))
wf.close()

print(f"\nDone")

# Record Audio as wav file - fixed time
### The python-sounddevice and pyaudio libraries provide ways to record audio with Python. python-sounddevice records to NumPy arrays and pyaudio records to bytes objects. Both of these can be stored as WAV files using the scipy and wave libraries, respectively.

In [6]:
import pyaudio
import wave
import os
import time
import contextlib

In [2]:
SAV2FILE = r'/home/rohit/PyWDUbuntu/thesis/audio/wavs/pyaudio_rec_3.wav'

In [3]:
DATA_APPEND_SECS = 3
LISTEN_SECS = 10

DEBUG_SWITCH = False

In [7]:
def listen_and_save_file(_specs_listen_list, _sav2File, _DEBUG_SWITCH):
    _append_every_secs, _max_secs  = _specs_listen_list
    
    ## setup recording parameters for deepspeech = 16kHz, mono channel, 16-bit, wav file for inference
    SAMPLE_FORMAT = pyaudio.paInt16  # 16 bits per sample
    CHANNELS = 1 # mono
    FS = 16000  # Record at 16kHz
    
    ## setup parameters for pyaudio
    CHUNCK = 1024  # Record in chunks of 1024 samples
    
    myStr = '\n'.join([
        f"\nListen parameters:",
        f"chunk = {CHUNCK}",
        f"channels = {CHANNELS}"
        f"sampling frequency = {FS} Hz",
        f"sample_format = 16 bit",
        f"append_every_secs = {_append_every_secs}",
        f"Listen Max secs = {_max_secs}"
    ])
    print(f"{myStr}")
    myStr = None
    
    ## Create an interface to PortAudio
    p = pyaudio.PyAudio()
    stream = p.open(format=SAMPLE_FORMAT,
                    channels=CHANNELS,
                    rate=FS,
                    frames_per_buffer=CHUNCK,
                    input=True) ## input = True means record
    
    ## Initialize array to store frames
    frames = []
    
    input(f"\nReady to record, press any key to begin....\n")
    start_time = time.time()
    print(f"\nStarted listening....\n")
    while True:
        for i in range(0, int(FS / CHUNCK * _append_every_secs)):
            data = stream.read(CHUNCK)
            frames.append(data)
            
            ## type(data) = <class 'bytes'>
            ## data =
            ## b'\x00\x00\xfe\xff\x01\x00\x05\x00\xed and continues for a long time
            ## data is a bytes array
            if _DEBUG_SWITCH:
                print(f"type(data) = {type(data)}")
                print(f"data =\n{data}")
                _DEBUG_SWITCH = False
        
        if (time.time() - start_time) > _max_secs:
            print(f"\nStopped recording: reached max recording time limit = {_max_secs} seconds")
            break
    
    # Stop and close the stream 
    stream.stop_stream()
    stream.close()
    # Terminate the PortAudio interface
    p.terminate()
    
    print(f"\nStopped listening....\n")
    
    # Save data to WAV file
    wf = wave.open(_sav2File, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(SAMPLE_FORMAT))
    wf.setframerate(FS)
    wf.writeframes(b''.join(frames))
    wf.close()
    
    ## get exact duration from saved wav file
    with contextlib.closing(wave.open(_sav2File,'r')) as f:
        in_frames = f.getnframes()
        in_rate = f.getframerate()
        rec_secs_exact = in_frames / float(in_rate)
    
    print(f"\nSaved to wav file: {_sav2File}")
    print(f"Duration = {rec_secs_exact:.2f} seconds\n")
    
    return

In [11]:
specs_listen_list = [
    DATA_APPEND_SECS,
    LISTEN_SECS
]

listen_and_save_file(specs_listen_list, SAV2FILE, DEBUG_SWITCH)

print(f"\nDone")


Listen parameters:
chunk = 1024
channels = 1sampling frequency = 16000 Hz
sample_format = 16 bit
append_every_secs = 3
Listen Max secs = 10

Ready to record, press any key to begin....


Started listening....


Stopped recording: reached max recording time limit = 10 seconds

Stopped listening....


Saved to wav file: /home/rohit/PyWDUbuntu/thesis/audio/wavs/pyaudio_rec_3.wav
Duration = 14.72 seconds


Done


## play the file just recorded in above cell

In [12]:
if os.path.exists(SAV2FILE):
    # Set chunk size of 1024 samples per data frame
    chunk = 1024  

    # Open the sound file 
    wf = wave.open(SAV2FILE, 'rb')

    # Create an interface to PortAudio
    p = pyaudio.PyAudio()

    # Open a .Stream object to write the WAV file to
    # 'output = True' indicates that the sound will be played rather than recorded
    stream = p.open(format = p.get_format_from_width(wf.getsampwidth()),
                    channels = wf.getnchannels(),
                    rate = wf.getframerate(),
                    output = True)

    # Read data in chunks
    data = wf.readframes(chunk)

    # Play the sound by writing the audio data to the stream
    ###########
    ## without checking for data = b'' it never exits so changed as per this link:
    # https://stackoverflow.com/questions/34856992/when-i-play-a-wav-in-python-the-program-doesnt-stop-when-the-file-ends-leavin
    #while data != '':
    #    stream.write(data)
    #    data = wf.readframes(chunk)
    ###########
    while data != b'':
        stream.write(data)
        data = wf.readframes(chunk)
    # Close and terminate the stream
    stream.close()
    p.terminate()
else:
    print(f"File not found: {SAV2FILE}")

print(f"\nDone")


Done


# Record Audio as wav file - fixed min time and then stop at silence or max time  - NOT WORKING AS EXPECTED

### Approach per: https://github.com/jeysonmc/python-google-speech-scripts/blob/master/stt_google.py   (link obtained from SO: https://stackoverflow.com/questions/19070290/pyaudio-listen-until-voice-is-detected-and-then-record-to-a-wav-file )
### The python-sounddevice and pyaudio libraries provide ways to record audio with Python. python-sounddevice records to NumPy arrays and pyaudio records to bytes objects. Both of these can be stored as WAV files using the scipy and wave libraries, respectively.

In [1]:
import pyaudio
import wave
import os
import datetime
import time
#import numpy as np
import contextlib
import struct

In [2]:
SAV2FILE = r'/home/rohit/PyWDUbuntu/thesis/audio/wavs/pyaudio_rec_3.wav'

In [26]:
DATA_APPEND_SECS = 3
LISTEN_MIN_SECS = 3
LISTEN_MAX_SECS = 12
#MIN_LEVEL_THRESHOLD = 200000000000000

DEBUG_SWITCH = False

In [42]:
def listen_and_save_file(_specs_listen_list, _sav2File, _DEBUG_SWITCH):
    #_append_every_secs, _min_secs, _max_secs, _min_threshold  = _specs_listen_list
    _append_every_secs, _min_secs, _max_secs  = _specs_listen_list
    
    ## setup recording parameters for deepspeech = 16kHz, mono channel, 16-bit, wav file for inference
    SAMPLE_FORMAT = pyaudio.paInt16  # 16 bits per sample
    CHANNELS = 1 # mono
    FS = 16000  # Record at 16kHz
    NUM_RECENT_FRAME_DATA_FOR_SILENCE = 100
    
    ## setup parameters for pyaudio
    CHUNCK = 1024  # Record in chunks of 1024 samples
    
    myStr = '\n'.join([
        f"\nListen parameters:",
        f"chunk = {CHUNCK}",
        f"channels = {CHANNELS}"
        f"sampling frequency = {FS} Hz",
        f"sample_format = 16 bit",
        f"append_every_secs = {_append_every_secs}",
        f"Listen Min secs = {_min_secs}",
        f"Listen Max secs = {_max_secs}"
    ])
    print(f"{myStr}")
    myStr = None
    
    ## Create an interface to PortAudio
    p = pyaudio.PyAudio()
    stream = p.open(format=SAMPLE_FORMAT,
                    channels=CHANNELS,
                    rate=FS,
                    frames_per_buffer=CHUNCK,
                    input=True) ## input = True means record
    
    ## Initialize array to store frames
    frames = []
    
    ## detect ambient noise level to set the minimum threshold value
    min_threshold = None
    input(f"\n\nPress any key to allow detecting ambient noise level.\nPlease DO NOT SPEAK.\n\n")
    start_time = time.time()
    for i in range(0, int(FS / CHUNCK * _append_every_secs)):
        data = stream.read(CHUNCK)
        frames.append(data)
        ## type(data) = <class 'bytes'>
        ## data =
        ## b'\x00\x00\xfe\xff\x01\x00\x05\x00\xed and continues for a long time
        ## data is a bytes array
    frames_all_data = []
    for bytearr in frames:
        for i in range(0, len(bytearr), 2):
            ## https://docs.python.org/3/library/struct.html
            ## =  for native endian, H for unsigned 2 byte short int, h for signed 2 byte short int
            frames_all_data.append( struct.unpack(r'=H', bytearr[i:i+2])[0] )
    min_threshold = sum([val*val for val in frames_all_data])
    ## increase by some factor
    min_threshold = int(1.20 * min_threshold)
    print(f"Threshold set to = {min_threshold}")
    print(f"Ambient noise detected for {(time.time() - start_time):.2f} seconds")
    print(f"\nNow main recording process can begin....\n\n")
    data, frames_all_data, frames = None, None, None
    
    ## Re-initialize array to store frames
    frames = []
    
    input(f"\nReady to record, press any key to begin....\n")
    start_time = time.time()
    print(f"\nStarted listening....\n")
    while True:
        for i in range(0, int(FS / CHUNCK * _append_every_secs)):
            data = stream.read(CHUNCK)
            frames.append(data)
            
            ## type(data) = <class 'bytes'>
            ## data =
            ## b'\x00\x00\xfe\xff\x01\x00\x05\x00\xed and continues for a long time
            ## data is a bytes array
            if _DEBUG_SWITCH:
                print(f"type(data) = {type(data)}")
                print(f"data =\n{data}")
                _DEBUG_SWITCH = False
        
        print(f"\ntime.time = {time.time()}\tstart_time = {start_time}\t_min_sec = {_min_secs}")
        print(f"{(time.time() - start_time) > _min_secs}")
        if (time.time() - start_time) > _min_secs:
            ## check if silence in last DATA_APPEND_SECS seconds
            frames_recent_all_data = []
            for bytearr in frames[-NUM_RECENT_FRAME_DATA_FOR_SILENCE:]:
                for i in range(0, len(bytearr), 2):
                    ## https://docs.python.org/3/library/struct.html
                    ## =  for native endian, H for unsigned 2 byte short int, h for signed 2 byte short int
                    frames_recent_all_data.append( struct.unpack(r'=H', bytearr[i:i+2])[0] )
            level_data = sum([val*val for val in frames_recent_all_data])
            print(f"level_data = {level_data}")
            frames_recent_all_data = None
            
            if level_data < min_threshold:
                print(f"\nStopped recording: as volume was below threshold....")
                print(f"level_data ({level_data}) < threshold value ({min_threshold})")
                break
        print(f"\ntime.time() = {time.time()}\tstart_time = {start_time}\t_max_sec = {_max_secs}")
        print(f"{(time.time() - start_time) > _max_secs}")
        if (time.time() - start_time) > _max_secs:
            print(f"\nStopped recording: reached max recording time limit = {_max_secs} seconds")
            break
    
    # Stop and close the stream 
    stream.stop_stream()
    stream.close()
    # Terminate the PortAudio interface
    p.terminate()
    
    rec_secs_approx = time.time() - start_time
    
    print(f"\nStopped listening....\n")
    
    # Save data to WAV file
    wf = wave.open(_sav2File, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(SAMPLE_FORMAT))
    wf.setframerate(FS)
    wf.writeframes(b''.join(frames))
    wf.close()
    
    ## get exact duration from saved wav file
    with contextlib.closing(wave.open(_sav2File,'r')) as f:
        in_frames = f.getnframes()
        in_rate = f.getframerate()
        rec_secs_exact = in_frames / float(in_rate)
    
    print(f"\nSaved to wav file: {_sav2File}")
    print(f"Duration approx. = {rec_secs_approx:.2f} seconds\nDuration exact. = {rec_secs_exact:.2f} seconds\n")
    
    return

In [43]:
specs_listen_list = [
    DATA_APPEND_SECS,
    LISTEN_MIN_SECS,
    LISTEN_MAX_SECS#,
    #MIN_LEVEL_THRESHOLD 
]

listen_and_save_file(specs_listen_list, SAV2FILE, DEBUG_SWITCH)

print(f"\nDone")


Listen parameters:
chunk = 1024
channels = 1sampling frequency = 16000 Hz
sample_format = 16 bit
append_every_secs = 3
Listen Min secs = 3
Listen Max secs = 12


Press any key to allow detecting ambient noise level.
Please DO NOT SPEAK.


Threshold set to = 119293501930388
Ambient noise detected for 1.21 seconds

Now main recording process can begin....



Ready to record, press any key to begin....


Started listening....


time.time = 1602627607.2429671	start_time = 1602627605.877094	_min_sec = 3
False

time.time() = 1602627607.2431293	start_time = 1602627605.877094	_max_sec = 12
False

time.time = 1602627610.1479814	start_time = 1602627605.877094	_min_sec = 3
True
level_data = 203638550721896

time.time() = 1602627610.1838055	start_time = 1602627605.877094	_max_sec = 12
False

time.time = 1602627613.162787	start_time = 1602627605.877094	_min_sec = 3
True
level_data = 227267871096094

time.time() = 1602627613.1979096	start_time = 1602627605.877094	_max_sec = 12
False

time.time = 16

## play the file just recorded in above cell

In [25]:
if os.path.exists(SAV2FILE):
    # Set chunk size of 1024 samples per data frame
    chunk = 1024  

    # Open the sound file 
    wf = wave.open(SAV2FILE, 'rb')

    # Create an interface to PortAudio
    p = pyaudio.PyAudio()

    # Open a .Stream object to write the WAV file to
    # 'output = True' indicates that the sound will be played rather than recorded
    stream = p.open(format = p.get_format_from_width(wf.getsampwidth()),
                    channels = wf.getnchannels(),
                    rate = wf.getframerate(),
                    output = True)

    # Read data in chunks
    data = wf.readframes(chunk)

    # Play the sound by writing the audio data to the stream
    ###########
    ## without checking for data = b'' it never exits so changed as per this link:
    # https://stackoverflow.com/questions/34856992/when-i-play-a-wav-in-python-the-program-doesnt-stop-when-the-file-ends-leavin
    #while data != '':
    #    stream.write(data)
    #    data = wf.readframes(chunk)
    ###########
    while data != b'':
        stream.write(data)
        data = wf.readframes(chunk)
    # Close and terminate the stream
    stream.close()
    p.terminate()
else:
    print(f"File not found: {SAV2FILE}")

print(f"\nDone")


Done


In [ ]:
""" ## backup
def listen_and_save_file(_specs_listen_list, _sav2File, _DEBUG_SWITCH):
    #_append_every_secs, _min_secs, _max_secs, _min_threshold  = _specs_listen_list
    _append_every_secs, _min_secs, _max_secs  = _specs_listen_list
    
    ## setup recording parameters for deepspeech = 16kHz, mono channel, 16-bit, wav file for inference
    SAMPLE_FORMAT = pyaudio.paInt16  # 16 bits per sample
    CHANNELS = 1 # mono
    FS = 16000  # Record at 16kHz
    
    ## setup parameters for pyaudio
    CHUNCK = 1024  # Record in chunks of 1024 samples
    
    myStr = '\n'.join([
        f"\nListen parameters:",
        f"chunk = {CHUNCK}",
        f"channels = {CHANNELS}"
        f"sampling frequency = {FS} Hz",
        f"sample_format = 16 bit",
        f"append_every_secs = {_append_every_secs}",
        f"Listen Min secs = {_min_secs}",
        f"Listen Max secs = {_max_secs}"
    ])
    print(f"{myStr}")
    myStr = None
    
    ## Create an interface to PortAudio
    p = pyaudio.PyAudio()
    stream = p.open(format=SAMPLE_FORMAT,
                    channels=CHANNELS,
                    rate=FS,
                    frames_per_buffer=CHUNCK,
                    input=True) ## input = True means record
    
    ## detect ambient noise level to set the minimum threshold value
    min_threshold = None
    input(f"\n\nPress any key to allow detecting ambient noise level.\nPlease DO NOT SPEAK for {_append_every_secs} seconds.\n\n")
    start_time = time.time()
    frames_append_every_secs = []
    for i in range(0, int(FS / CHUNCK * _append_every_secs)):
        data = stream.read(CHUNCK)
        frames_append_every_secs.append(data)
    frames_append_every_secs_all_data = []
    for bytearr in frames_append_every_secs:
        for i in range(0, len(bytearr), 2):
            ## https://docs.python.org/3/library/struct.html
            ## =  for native endian, H for unsigned 2 byte short int, h for signed 2 byte short int
            frames_append_every_secs_all_data.append( struct.unpack(r'=H', bytearr[i:i+2])[0] )
    min_threshold = sum([val*val for val in frames_append_every_secs_all_data])
    ## increase by some factor
    min_threshold = int(1.20 * min_threshold)
    print(f"Threshold set to = {min_threshold}")
    print(f"Ambient noise detected for {(time.time() - start_time):.2f} seconds")
    print(f"\nNow main recording process can begin....\n\n")
    del data, frames_append_every_secs_all_data, frames_append_every_secs
    
    ## Initialize array to store frames
    frames = []
    
    input(f"\nPress any key to begin process....\n")
    start_time = time.time()
    print(f"\nStarted listening....\n")
    while True:
        frames_append_every_secs = []
        for i in range(0, int(FS / CHUNCK * _append_every_secs)):
            data = stream.read(CHUNCK)
            frames.append(data)
            frames_append_every_secs.append(data)
            
            ## type(data) = <class 'bytes'>
            ## data =
            ## b'\x00\x00\xfe\xff\x01\x00\x05\x00\xed and continues for a long time
            ## data is a bytes array
            if _DEBUG_SWITCH:
                print(f"type(data) = {type(data)}")
                print(f"data =\n{data}")
                _DEBUG_SWITCH = False
        
        ## check if silence in last DATA_APPEND_SECS seconds
        frames_append_every_secs_all_data = []
        for bytearr in frames_append_every_secs:
            for i in range(0, len(bytearr), 2):
                ## https://docs.python.org/3/library/struct.html
                ## =  for native endian, H for unsigned 2 byte short int, h for signed 2 byte short int
                frames_append_every_secs_all_data.append( struct.unpack(r'=H', bytearr[i:i+2])[0] )
        """
        volmax = max(frames_all_data)
        volmin = min(frames_all_data)
        print(f"max volume = {volmax}")
        print(f"min volume = {volmin}")
        print(f"frames_all_data = {frames_all_data[:100]}")
        """
        level_data = sum([val*val for val in frames_append_every_secs_all_data])
        print(f"level_data = {level_data}")
        del frames_append_every_secs
        
        if (level_data < min_threshold) and ( (time.time() - start_time) > _min_secs ):
            print(f"\nStopped recording: in last {_append_every_secs} seconds, volume was below threshold")
            print(f"level_data ({level_data}) < threshold value ({min_threshold})")
            break
        elif (time.time() - start_time) > _max_secs:
            print(f"\nStopped recording: reached max recording time limit = {_max_secs} seconds")
            break
    
    # Stop and close the stream 
    stream.stop_stream()
    stream.close()
    # Terminate the PortAudio interface
    p.terminate()
    
    rec_secs_approx = time.time() - start_time
    
    print(f"\nStopped listening....\n")
    
    # Save data to WAV file
    wf = wave.open(_sav2File, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(SAMPLE_FORMAT))
    wf.setframerate(FS)
    wf.writeframes(b''.join(frames))
    wf.close()
    
    ## get exact duration from saved wav file
    with contextlib.closing(wave.open(_sav2File,'r')) as f:
        in_frames = f.getnframes()
        in_rate = f.getframerate()
        rec_secs_exact = in_frames / float(in_rate)
    
    print(f"\nSaved to wav file: {_sav2File}")
    print(f"Duration approx. = {rec_secs_approx:.2f} seconds\nDuration exact. = {rec_secs_exact:.2f} seconds\n")
    
    return
"""